In [2]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
import matplotlib.gridspec as gridspec
cudnn.benchmark = True
from sklearn.metrics import confusion_matrix

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [4]:
device = torch.device("cpu")

In [5]:
model_ft = models.resnet50()


num_ftrs = model_ft.fc.in_features


model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)


model_ft.load_state_dict(torch.load('weights-resnet.pth',map_location ='cpu'))


<All keys matched successfully>

In [6]:
nb_classes = 10

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device=device)
lbllist=torch.zeros(0,dtype=torch.long, device=device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        from sklearn.metrics import classification_report

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

print(classification_report(lbllist.numpy(), predlist.numpy()))

              precision    recall  f1-score   support

           0       0.71      0.78      0.74        99
           1       0.70      0.66      0.68       129
           2       0.91      0.84      0.87       147
           3       0.72      0.81      0.77       165
           4       1.00      0.69      0.81        64
           5       1.00      0.81      0.90        43
           6       0.76      0.76      0.76       177
           7       1.00      0.79      0.88        28
           8       0.66      0.80      0.72       162
           9       0.98      0.87      0.92        69

    accuracy                           0.78      1083
   macro avg       0.85      0.78      0.81      1083
weighted avg       0.79      0.78      0.78      1083



In [ ]:
from sklearn.metrics import classification_report
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')
with torch.no_grad():
     for i, (inputs, classes) in enumerate(dataloaders['val']):
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])
print(classification_report(lbllist.numpy(), predlist.numpy(), target_names=class_names, digits=2))